TODO: 
1. Load and restructure data. 
2. Restructure sentence boundaries. 
3. Run Phrases model 
4. Run CBOW model 
5. Assess CBOW model

# 0. Load and reformat data

In [1]:
import gensim

In [5]:
import json 

path = r"C:\\Users\\bened\\DataScience\\ANLP\\AT2\\preprocessed_data\\screenplays_preprocessed.json"

with open(path, 'r') as f:
    data = json.load(f)

In [62]:
print(len(data))

1995


In [6]:
print(data["0"][:10])

['@1', 'night', 'write', 'steve', 'koren', 'chris', 'kattan', 'june', 'panoramic', 'view']


In [7]:
screenplay_data = {int(k): v for k, v in data.items()}

In [63]:
print(len(screenplay_data))

1995


In [74]:
unique_tokens = set(
    token for tokens in screenplay_data.values() for token in tokens
)
vocab_size = len(unique_tokens)
print(vocab_size)

242452


In [8]:
print(screenplay_data[10][:10])

['@0', 'last', 'mohicans', 'write', 'michael', 'mann', 'christopher', 'crowe', '\n ', 'screen']


In [9]:
import copy 

mo = copy.deepcopy(screenplay_data[10])
for i, tok in enumerate(mo):
    if i == 50:
        break
    else:
        print(tok)

@0
last
mohicans
write
michael
mann
christopher
crowe

 
screen
leaf
crystal
drop
precipitation
landscape
miniature
hear
forest
distant
bird
sound
seem
piece
sunlight
refract
drop
water
paint
patch
moss
yellow
whisper
wind
join
sound
mix
distant
rustling
get
close
louder
shallow
breathing
get
ominous
interloper
floor
forest
come
suddenly


In [6]:
for t in mo:
    if t.startswith("@"):
        print(t)

@0
@1
@2
@1
@1
@0
@1
@1
@1
@1
@1
@1
@2
@1
@1
@0
@1
@2
@2
@1
@1
@2
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@0
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@2
@1
@0
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@2
@1
@2
@1
@2
@1
@2
@2
@2
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@0
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@0
@0
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@0
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@0
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@0
@2
@1
@1
@1
@1
@1
@2
@2
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1


In [10]:
# search for @ and \n
counter = 0
for i, t in enumerate(mo):
    if counter == 20:
        break
    if t.startswith("\n"):
        print(mo[i-5:i+5])
        counter += 1

['write', 'michael', 'mann', 'christopher', 'crowe', '\n ', 'screen', 'leaf', 'crystal', 'drop']
['heavy', 'even', 'breathe', 'learn', 'later', '\n ', '@2', ':', 'man', 'unca']
['branch', 'disturb', 'make', 'sound', 'tan', '\n ', 'two', 'hundred', 'fifty', 'yards']
['caliber', 'round', 'program', '.', 'moment', '\n ', '@2', ':', 'impact', '@1']
['long', 'knife', 'approach', 'fall', 'elk', '\n ', '@0', 'exterior', 'interior', 'cameron']
['hand', 'fence', 'chingachgook', 'john', 'cameron', '\n ', 'doorway', 'cameron', 'interior', 'set']
['good', 'another', 'year', 'pass', 'john', '\n ', 'gettin', 'along', 'yes', 'warm']
['heart', 'farm', 'good', 'year', 'john', '\n ', 'good', 'year', 'corn', 'mohawk']
['take', 'much', 'fur', 'horicane', 'sic', '\n ', '@1', 'trap', 'tradin', 'castleton']
['\n ', '@1', 'trap', 'tradin', 'castleton', '\n ', 'schylerville', 'dutch', 'french', 'english']
['mohican', 'child', 'leave', 'brother', 'peace', '\n ', 'laugh', 'hawkeye', 'chingachgook', '\n ']
['peac

I've now loaded my preprocessed data into a new notebook and want to prepare it for a gensim Phrases model. The Phrases module takes lists of word-tokenized sentences as input (I believe?).   Currently my data is structured like this: 
{screenplay_index(int):['@n', 't1', t2', ..., 'ti', '\n'

Where n is a label 0, 1 or 2 that provides information on the line

In [11]:
# search for @ and \n
counter = 0
for i, t in enumerate(mo):
    if counter == 20:
        break
    if t.startswith(r"@") or t.startswith(r"\\n"):
        print(mo[i-1:i+2])
        counter += 1


[]
['foot', '@1', 'rocket']
['\n ', '@2', ':']
['day', '@1', 'rocket']
['tribe', '@1', 'shirt']
['foliage', '@0', 'suddenly']
['shoulder', '@1', 'cock']
['sound', '@1', 'slip']
['sight', '@1', 'five']
['tight', '@1', 'hold']
['sound', '@1', 'crack']
['lightning', '@1', 'leap']
['\n ', '@2', ':']
['impact', '@1', 'three']
['worldliness', '@1', 'anglo']
['\n ', '@0', 'exterior']
['\n ', '@1', 'trap']
['\n ', '@2', ':']
['\n ', '@2', ':']
['fast', '@1', 'laugh']


# Prepare Data for Vectorization

Desire data structure: 
{
    label:
    {sentence_vector:
        {word_vectors}
    }
}

As sentences:
{label: [sentences[sent1][sent2][etc.]]}


In [79]:
# find and return all non-word chars in the data  
import re

allowed = {'@', '.', ':', '\n', '-'}

filtered_tokens = set()

for tokens in screenplay_data.values():
    for token in tokens:
        if any(
            char for char in token if re.search(r'\W', char) and char not in allowed
        ):
            filtered_tokens.add(token)

print(filtered_tokens)

{'â\x80\x98yhemselve', "i's", 'â\x80\x9cwhoa.â\x80\x9d', 'elementaryâ\x80\x94', 'â\x80\x98ereâ\x80\x99s', 'â\x80\x98behold', 'â\x80\x9cadversity', 'half=-speed', 'â\x80\x9ctalent.â\x80\x9d', 'originaâ\x80\x99', '\\uj', '\x00s\x00c\x00r\x00e\x00w\x00', 'â\x80\x98unpack', 'â\x80\x98pal', 'then-~', 'vinodâ\x80\x99s', '\x00a\x00l\x00m\x00o\x00s\x00t\x00', '\x00p\x00a\x00s\x00s\x00e\x00s\x00', "1980's", 'itâ\x80\x99li', 'thenâ\x80\x94look', "n'stupid", 'palâ\x80\x9d', 'â\x80\x9criele', 'elseâ\x80\x99s', 'â\x80\x98union', 'fireyâ\x80\x99s', 'tailorâ\x80\x99s', 'â\x80\x9clibrary', 'messâ\x80\x99', '/president', 'donâ\x80\x99', 'emaâ\x80\x99s', 'itchy.â\x80\x9d', 'sisterâ\x80\x9d', "you'ii", 'battle.â\x80\x9d', 'â\x80\x9csc', 'greens.â\x80\x9d', 'folly|of', 'â\x80\x98nevertheless', 'â\x80\x98ficial', '\x00d\x00o\x00w\x00n\x00.\x00', 'â\x80\x9cllewyn', '\x00d\x00u\x00r\x00i\x00n\x00g\x00', 'dreaminâ\x80\x99', 'lightsâ\x80\x99', 'whoâ\x80\x99ve', 'â\x80\x9ckidâ\x80\x9d.', 'huntsmanâ\x80\x99s', '

In [80]:
print(len(filtered_tokens))

37208


In [75]:
import numpy as np
import re

def preprocess_for_phrases(data):

    labeled_sentences = {}

    for screenplay_idx, tokens in data.items():

        current_sentence = []
        processed_line = []
        line_label = None

        for token in tokens:
            
            # parse label
            if token.startswith('@'):
                line_label = np.int8(token[1:])
                labeled_sentences[line_label] = []

            # skip if token contains non-word chars 
            elif re.search(r'\W', token) and token not in {'.', '\n'}:
                continue

            elif token == ':':
                continue

            # skip over '-' 
            elif token == '-':
                continue
            
            elif token == '.':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            elif token == '\n':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            else:
                current_sentence.append(token)
        
        if current_sentence:
            processed_line.append(current_sentence)
        
        if line_label is not None:
            labeled_sentences[line_label].extend(processed_line)
        
    return labeled_sentences


In [76]:
prepared_data = preprocess_for_phrases(screenplay_data)

In [77]:
sents = [sent for sentences in prepared_data.values() for sent in sentences]

In [78]:
unique_tokens = set(token for sentence in sents for token in sentence)
vocabulary_size = len(unique_tokens)
print(vocabulary_size)

2240


# Phrases

In [21]:
# ! pip install --upgrade numpy gensim

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [25]:
# import numpy 
# import gensim
# import scipy

# print(numpy.__version__)
# print(gensim.__version__)
# print(scipy.__version__)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [15]:
from gensim.models import Word2Vec, phrases

In [45]:
bigrams = phrases.Phrases(
    sentences=sents,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS
)
bigram_model = phrases.Phraser(bigrams)
bigrams_dict = bigrams.export_phrases()

In [ ]:
print(len())

In [18]:
! pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [43]:
import pandas as pd

bigrams_df = pd.DataFrame.from_dict(
    bigrams_dict, 
    orient='index'
)
bigrams_df.reset_index(inplace=True)
bigrams_df.columns = ["phrase", "score"]
bigrams_df.sort_values('score', inplace=True, ascending=False)
print(bigrams_df.head(10))
print(bigrams_df.tail(10))

               phrase       score
16               8_yr  355.200000
1        baseball_cap  296.000000
2           duct_tape  253.714286
17             yr_old  170.769231
15          towel_boy   99.495798
19           sir_well   87.058824
18           well_sir   87.058824
12         rental_car   82.222222
4   service_stairwell   80.727273
6         view_mirror   79.285714
            phrase      score
0   goldenrod_page  76.917563
9      living_room  74.000000
3       cell_phone  74.000000
5        rear_view  55.966387
11    coffee_table  37.000000
14        car_park  28.190476
8      house_night  20.000000
13    basement_car  17.619048
7     mother_house  14.800000
10       day_later  13.393665


### Trigrams

Plausible trigrams like "x_day_later", "x_yr_old", so we'll run for trigrams. 

In [46]:
trigrams = phrases.Phrases(bigrams[sents])
trigram_model = phrases.Phraser(trigrams)
trigrams_dict = trigrams.export_phrases()
trigrams_df = pd.DataFrame.from_dict(
    trigrams_dict, 
    orient='index'
)
trigrams_df.reset_index(inplace=True)
trigrams_df.columns = ["phrase", "score"]
trigrams_df.sort_values('score', inplace=True, ascending=False)
print(trigrams_df.head(10))
print(trigrams_df.tail(10))


               phrase         score
0      goldenrod_page  48410.062500
1        baseball_cap  13354.500000
14           8_yr_old   1424.480000
10         rental_car    692.455556
6         living_room    556.437500
2   service_stairwell    445.150000
13          towel_boy    370.958333
12  basement_car_park    342.423077
4    rear_view_mirror    247.305556
3           rear_view    148.383333
                phrase       score
2    service_stairwell  445.150000
13           towel_boy  370.958333
12   basement_car_park  342.423077
4     rear_view_mirror  247.305556
3            rear_view  148.383333
11        basement_car  123.652778
9         coffee_table  105.988095
8   goldenrod_page_joe   49.324100
5         mother_house   29.976431
7            day_later   24.526171


much better

In [47]:
folder_path = r"C:\\Users\\bened\\DataScience\\ANLP\\AT2\\models"
trigram_model.save(f"{folder_path}\\trigram_model.pkl")

In [54]:
cbow_model = Word2Vec(
    sentences=trigrams[sents],
    vector_size=64,
    window=8,
    sg=0
)

In [55]:
word_vectors = cbow_model.wv 
print(len(word_vectors))
# word_vectors.save(f"{folder_path}\\cbow3_vectors.wordvectors")

354


# Vector Evaluation

In [51]:
def find_synonyms(vectors, word, topn=5):
    synonyms = vectors.most_similar(word, topn=topn)
    print(f"Words most similar to {word}:")
    for w, s in synonyms:
        print(f'{w}: {s:.4f}')

In [53]:
print(len(word_vectors))

354


In [52]:
find_synonyms(word_vectors, 'fuck', 5)

KeyError: "Key 'fuck' not present in vocabulary"